# 개선된 Vector 객체
## 사용자 정의 시퀀스형

In [1]:

from array import array
import math
import reprlib

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [3]:
Vector((3,4,5))

Vector([3.0, 4.0, 5.0])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 프로토콜과 덕 타이핑
### 파이썬에서 프로토콜은 문서에만 정의되는 개념이다.
### 예를 들어 시퀀스 프로토콜을 따른다면 단지 특별 메소드로 len 과 getitem을 정의하면 된다.
### 이를 문법상으로 프토토콜을 따르고 있다는 것을 보여주지 않아도 된다. 따라서 프로토콜 전부를 구현할 필요도 없다.
### 이러한 매커니즘을 덕 타이핑이라 한다.

In [5]:
# 시퀀스 프로토콜을 따르는 객체
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck :
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self) :
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self) :
        return len(self._cards)
    
    def __getitem__(self, position) :
        return self._cards[position]
    
deck = FrenchDeck()
print(len(deck))
print(deck[2:5])

52
[Card(rank='4', suit='spades'), Card(rank='5', suit='spades'), Card(rank='6', suit='spades')]


## Vector 버전 #2 : 슬라이스 가능한 시퀀스

In [6]:

from array import array
import math
import numbers
import reprlib
from typing import Type

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    def __len__(self) :
        return len(self._components)
    
    # 배열이 아니라 Vector 객체를 반환하게 하기
    def __getitem__(self, index) :
        cls = type(self)
        # 시퀀스 객체라면 Vector 객체를 반환한다.
        if isinstance(index, slice) :
            return cls(self._components[index])
        # indexing 이라면 Vector 객체를 반환하지 않고 인수만 반환한다.
        elif isinstance(index, numbers.Integral) :
            return self._components[index]
        else :
            msg = f'{cls.__name__} indices must be integers'
            raise TypeError(msg)
        
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [7]:
v1 = Vector([3,4,5])
len(v1)

3

In [8]:
v1[0], v1[-1]

(3.0, 5.0)

In [9]:
v7 = Vector(range(7))
v7[1:4]

Vector([1.0, 2.0, 3.0])

## Vector 버전 #3 : 동적 속성 접근

### \_\_getattr__
- my_obj.x 표현식이 주어지면 my_obj에 x 속성이 있는지 검사한다. 속성이 없으면 삭송 그래프를 따로 올라가면서 검사한다.
- 그래도 못 찾으면 \_\_get_attr__ 을 호출한다.

In [10]:

from array import array
import math
import reprlib

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name) :
        cls = type(self)
        if len(name) == 1 :
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components) : 
                return self._components[pos]
        msg = f"{self.__name__} object has no attribute {name}"
        raise AttributeError(msg) 
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [11]:
v = Vector(range(5))
print(v.x, v.y, v.z, v.t)

0.0 1.0 2.0 3.0


In [12]:
# 발생할 수 있는 에러

# v.x 에 10을 할당한다.
v.x = 10
# v.x 에는 10이 저장되어있다.
print(v.x)
# 하지만 v에는 0 이다.
print(v)

# v.x에 값이 새롭게 할당되면서 v.x는 v[0] 과 구분되는 새로운 속성이 되어버린것이다.

10
(0.0, 1.0, 2.0, 3.0, 4.0)


## \_\_setattr__

In [13]:

from array import array
import math
import reprlib

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name) :
        cls = type(self)
        if len(name) == 1 :
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components) : 
                return self._components[pos]
        msg = f"{cls.__name__} object has no attribute {name}"
        raise AttributeError(msg) 
        
    # 생성자와 같은 기타 모든 속성을 할당할 때 사용되므로 주의하자
    def __setattr__(self, name, value) :
        cls = type(self)
        if len(name) == 1 :
            if name in cls.shortcut_names :
                error = f'readonly attribute {name}'
            elif name.islower() :
                error = f"can't set attributes 'a' to 'z' in {cls.__name__}"
            else :
                error = ''
            if error :
                raise AttributeError(error)
            
        # super()은 메타 Vector ..?? 
        super().__setattr__(name, value)
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [14]:
v = Vector([1,2,3])
print(v.x)

try : 
    v.x = 10
except AttributeError as e :
    print(e)

1.0
readonly attribute x


## Vector 버전 #4 : 해싱 및 더 빠른 ==

In [19]:

from array import array
import math
import reprlib
import functools
import operator

class Vector :
    typecode ='d'

    # 시퀀스형 객체를 입력받아서 Vector를 생성하기.
    def __init__(self, components) :
        self._components = array(self.typecode, components)
    
    def __iter__(self) :
        return iter(self._components)
    
    def __len__(self) :
        return len(self._components)
        
    def __repr__(self) :
        # reprlib 은 components 를 제한된 길이로 출력한다.
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f"Vector({components})"
    
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name) :
        cls = type(self)
        if len(name) == 1 :
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components) : 
                return self._components[pos]
        msg = f"{cls.__name__} object has no attribute {name}"
        raise AttributeError(msg) 
        
    # 생성자와 같은 기타 모든 속성을 할당할 때 사용되므로 주의하자
    def __setattr__(self, name, value) :
        cls = type(self)
        if len(name) == 1 :
            if name in cls.shortcut_names :
                error = f'readonly attribute {name}'
            elif name.islower() :
                error = f"can't set attributes 'a' to 'z' in {cls.__name__}"
            else :
                error = ''
            if error :
                raise AttributeError(error)
            
        # super()은 메타 Vector ..?? 
        super().__setattr__(name, value)
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(self._components)
    
    def __eq__(self, other) :
        return len(self) == len(other) and all(a == b for a,b in zip(self._components, other._components))
    
    def __hash__(self) :
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
        
    
    # 유클리드 거리
    def __abs__(self) :
        return math.sqrt(sum(x * x for x in self))
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [22]:
Vector([3,4]) == Vector([3,4])

True